# Exploratory Data Analysis - Rossmann Store Sales

**Agent**: DataExplorer
**Date**: 2025-11-06
**Objective**: Comprehensive analysis of training data to understand patterns, data quality, and inform feature engineering

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Data Loading and Initial Inspection

In [ ]:
# Load the training data
train = pd.read_csv('../data/rossmann-store-sales/train.csv', parse_dates=['Date'])

print(f"Dataset Shape: {train.shape}")
print(f"Number of Stores: {train['Store'].nunique()}")
print(f"Date Range: {train['Date'].min()} to {train['Date'].max()}")
print(f"Total Days: {(train['Date'].max() - train['Date'].min()).days}")

In [ ]:
# Display first few rows
train.head(10)

In [ ]:
# Data types and structure
print("Data Types:")
print(train.dtypes)
print("\nData Info:")
train.info()

## 2. Missing Values Analysis

In [ ]:
# Check for missing values
missing = train.isnull().sum()
missing_pct = (missing / len(train)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Percentage': missing_pct
}).sort_values('Missing_Count', ascending=False)

print("Missing Values Summary:")
print(missing_df[missing_df['Missing_Count'] > 0])

if missing_df['Missing_Count'].sum() == 0:
    print("\n✓ No missing values found in the dataset!")

## 3. Descriptive Statistics

In [ ]:
# Numerical features statistics
print("Numerical Features Statistics:")
train.describe()

In [ ]:
# Categorical features
categorical_cols = ['DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
print("Categorical Features Summary:")
for col in categorical_cols:
    print(f"\n{col}:")
    print(train[col].value_counts().sort_index())

## 4. Target Variable Analysis (Sales)

In [ ]:
# Sales distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histogram
axes[0, 0].hist(train['Sales'], bins=100, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Sales Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Sales')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(train['Sales'].mean(), color='red', linestyle='--', label=f'Mean: {train["Sales"].mean():.2f}')
axes[0, 0].axvline(train['Sales'].median(), color='green', linestyle='--', label=f'Median: {train["Sales"].median():.2f}')
axes[0, 0].legend()

# Box plot
axes[0, 1].boxplot(train['Sales'])
axes[0, 1].set_title('Sales Box Plot', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Sales')

# Log-transformed histogram
sales_nonzero = train[train['Sales'] > 0]['Sales']
axes[1, 0].hist(np.log1p(sales_nonzero), bins=100, edgecolor='black', alpha=0.7, color='orange')
axes[1, 0].set_title('Log-Transformed Sales Distribution (Sales > 0)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Log(Sales + 1)')
axes[1, 0].set_ylabel('Frequency')

# Q-Q plot
from scipy import stats
stats.probplot(sales_nonzero, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot (Sales > 0)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../docs/sales_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Sales Statistics:")
print(f"  Mean: {train['Sales'].mean():.2f}")
print(f"  Median: {train['Sales'].median():.2f}")
print(f"  Std Dev: {train['Sales'].std():.2f}")
print(f"  Min: {train['Sales'].min():.2f}")
print(f"  Max: {train['Sales'].max():.2f}")
print(f"  Skewness: {train['Sales'].skew():.2f}")
print(f"  Kurtosis: {train['Sales'].kurtosis():.2f}")
print(f"\nZero Sales Records: {(train['Sales'] == 0).sum()} ({(train['Sales'] == 0).sum() / len(train) * 100:.2f}%)")

## 5. Store-Level Analysis

In [ ]:
# Sales by store
store_stats = train.groupby('Store').agg({
    'Sales': ['mean', 'median', 'std', 'min', 'max', 'count'],
    'Customers': ['mean', 'median']
}).round(2)

print("Store-Level Statistics (Top 10 by Average Sales):")
print(store_stats.sort_values(('Sales', 'mean'), ascending=False).head(10))

# Visualize store performance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

store_avg_sales = train.groupby('Store')['Sales'].mean().sort_values(ascending=False)
axes[0].bar(range(len(store_avg_sales)), store_avg_sales.values, alpha=0.7)
axes[0].set_title('Average Sales by Store (Sorted)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Store Rank')
axes[0].set_ylabel('Average Sales')
axes[0].axhline(store_avg_sales.mean(), color='red', linestyle='--', label='Overall Mean')
axes[0].legend()

# Sales variance by store
store_std = train.groupby('Store')['Sales'].std().sort_values(ascending=False)
axes[1].bar(range(len(store_std)), store_std.values, alpha=0.7, color='orange')
axes[1].set_title('Sales Standard Deviation by Store (Sorted)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Store Rank')
axes[1].set_ylabel('Sales Std Dev')

plt.tight_layout()
plt.savefig('../docs/store_performance.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Temporal Patterns Analysis

In [ ]:
# Create temporal features
train['Year'] = train['Date'].dt.year
train['Month'] = train['Date'].dt.month
train['Day'] = train['Date'].dt.day
train['WeekOfYear'] = train['Date'].dt.isocalendar().week
train['Quarter'] = train['Date'].dt.quarter

# Sales over time
fig, axes = plt.subplots(3, 2, figsize=(16, 14))

# Daily sales trend
daily_sales = train.groupby('Date')['Sales'].mean().reset_index()
axes[0, 0].plot(daily_sales['Date'], daily_sales['Sales'], alpha=0.6)
axes[0, 0].set_title('Average Daily Sales Over Time', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Average Sales')
axes[0, 0].tick_params(axis='x', rotation=45)

# Sales by day of week
dow_sales = train.groupby('DayOfWeek')['Sales'].mean()
axes[0, 1].bar(dow_sales.index, dow_sales.values, color='skyblue', alpha=0.7)
axes[0, 1].set_title('Average Sales by Day of Week', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Day of Week (1=Mon, 7=Sun)')
axes[0, 1].set_ylabel('Average Sales')
axes[0, 1].set_xticks(range(1, 8))

# Sales by month
monthly_sales = train.groupby('Month')['Sales'].mean()
axes[1, 0].bar(monthly_sales.index, monthly_sales.values, color='lightgreen', alpha=0.7)
axes[1, 0].set_title('Average Sales by Month', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Average Sales')
axes[1, 0].set_xticks(range(1, 13))

# Sales by year
yearly_sales = train.groupby('Year')['Sales'].mean()
axes[1, 1].bar(yearly_sales.index, yearly_sales.values, color='coral', alpha=0.7)
axes[1, 1].set_title('Average Sales by Year', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Average Sales')

# Weekly trend
weekly_sales = train.groupby('WeekOfYear')['Sales'].mean()
axes[2, 0].plot(weekly_sales.index, weekly_sales.values, marker='o', markersize=3, alpha=0.6)
axes[2, 0].set_title('Average Sales by Week of Year', fontsize=14, fontweight='bold')
axes[2, 0].set_xlabel('Week of Year')
axes[2, 0].set_ylabel('Average Sales')
axes[2, 0].grid(True, alpha=0.3)

# Quarterly pattern
quarterly_sales = train.groupby('Quarter')['Sales'].mean()
axes[2, 1].bar(quarterly_sales.index, quarterly_sales.values, color='purple', alpha=0.7)
axes[2, 1].set_title('Average Sales by Quarter', fontsize=14, fontweight='bold')
axes[2, 1].set_xlabel('Quarter')
axes[2, 1].set_ylabel('Average Sales')
axes[2, 1].set_xticks(range(1, 5))

plt.tight_layout()
plt.savefig('../docs/temporal_patterns.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Feature Relationships and Correlations

In [ ]:
# Correlation matrix
numerical_features = ['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo', 'SchoolHoliday']
corr_matrix = train[numerical_features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, fmt='.3f', vmin=-1, vmax=1)
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../docs/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("Strongest correlations with Sales:")
print(corr_matrix['Sales'].sort_values(ascending=False))

## 8. Promotional and Holiday Effects

In [ ]:
# Promo effect
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Promo vs No Promo
promo_sales = train.groupby('Promo')['Sales'].mean()
axes[0, 0].bar(['No Promo', 'Promo'], promo_sales.values, color=['lightcoral', 'lightgreen'], alpha=0.7)
axes[0, 0].set_title('Average Sales: Promo vs No Promo', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Average Sales')
for i, v in enumerate(promo_sales.values):
    axes[0, 0].text(i, v + 100, f'{v:.0f}', ha='center', fontweight='bold')

# School Holiday effect
school_holiday_sales = train.groupby('SchoolHoliday')['Sales'].mean()
axes[0, 1].bar(['No Holiday', 'School Holiday'], school_holiday_sales.values, 
               color=['lightblue', 'lightyellow'], alpha=0.7)
axes[0, 1].set_title('Average Sales: School Holiday Effect', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Average Sales')
for i, v in enumerate(school_holiday_sales.values):
    axes[0, 1].text(i, v + 100, f'{v:.0f}', ha='center', fontweight='bold')

# State Holiday effect
state_holiday_sales = train.groupby('StateHoliday')['Sales'].mean().sort_values(ascending=False)
axes[1, 0].bar(range(len(state_holiday_sales)), state_holiday_sales.values, alpha=0.7)
axes[1, 0].set_title('Average Sales by State Holiday Type', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('State Holiday (0=None, a=Public, b=Easter, c=Christmas)')
axes[1, 0].set_ylabel('Average Sales')
axes[1, 0].set_xticks(range(len(state_holiday_sales)))
axes[1, 0].set_xticklabels(state_holiday_sales.index)

# Open vs Closed
open_sales = train.groupby('Open')['Sales'].mean()
axes[1, 1].bar(['Closed', 'Open'], open_sales.values, color=['red', 'green'], alpha=0.7)
axes[1, 1].set_title('Average Sales: Open vs Closed', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('Average Sales')
for i, v in enumerate(open_sales.values):
    axes[1, 1].text(i, v + 100, f'{v:.0f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../docs/promotional_effects.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nPromo Effect Analysis:")
print(f"  Average Sales (No Promo): {promo_sales[0]:.2f}")
print(f"  Average Sales (Promo): {promo_sales[1]:.2f}")
print(f"  Promo Lift: {((promo_sales[1] / promo_sales[0] - 1) * 100):.2f}%")

## 9. Customer Behavior Analysis

In [ ]:
# Sales per customer
train['SalesPerCustomer'] = train['Sales'] / train['Customers']
train['SalesPerCustomer'] = train['SalesPerCustomer'].replace([np.inf, -np.inf], np.nan)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Sales vs Customers scatter
sample = train[train['Open'] == 1].sample(min(5000, len(train)))
axes[0, 0].scatter(sample['Customers'], sample['Sales'], alpha=0.3, s=10)
axes[0, 0].set_title('Sales vs Number of Customers', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Customers')
axes[0, 0].set_ylabel('Sales')
axes[0, 0].grid(True, alpha=0.3)

# Sales per customer distribution
spc_data = train[(train['Open'] == 1) & (train['SalesPerCustomer'].notna())]['SalesPerCustomer']
axes[0, 1].hist(spc_data, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Sales Per Customer Distribution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Sales Per Customer')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(spc_data.mean(), color='red', linestyle='--', label=f'Mean: {spc_data.mean():.2f}')
axes[0, 1].legend()

# Customers by day of week
dow_customers = train[train['Open'] == 1].groupby('DayOfWeek')['Customers'].mean()
axes[1, 0].bar(dow_customers.index, dow_customers.values, color='teal', alpha=0.7)
axes[1, 0].set_title('Average Customers by Day of Week', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Day of Week (1=Mon, 7=Sun)')
axes[1, 0].set_ylabel('Average Customers')
axes[1, 0].set_xticks(range(1, 8))

# Sales per customer by promo
spc_promo = train[(train['Open'] == 1) & (train['SalesPerCustomer'].notna())].groupby('Promo')['SalesPerCustomer'].mean()
axes[1, 1].bar(['No Promo', 'Promo'], spc_promo.values, color=['orange', 'green'], alpha=0.7)
axes[1, 1].set_title('Sales Per Customer: Promo Effect', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('Sales Per Customer')
for i, v in enumerate(spc_promo.values):
    axes[1, 1].text(i, v + 0.5, f'{v:.2f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../docs/customer_behavior.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nCustomer Behavior Insights:")
print(f"  Average Sales Per Customer: ${spc_data.mean():.2f}")
print(f"  Median Sales Per Customer: ${spc_data.median():.2f}")
print(f"  Correlation (Sales, Customers): {train['Sales'].corr(train['Customers']):.3f}")

## 10. Outlier Detection

In [ ]:
# Identify outliers using IQR method
def detect_outliers_iqr(data, column, multiplier=1.5):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Sales outliers (only for open stores)
open_stores = train[train['Open'] == 1]
sales_outliers, sales_lower, sales_upper = detect_outliers_iqr(open_stores, 'Sales')

print(f"Sales Outliers (Open Stores):")
print(f"  Lower Bound: {sales_lower:.2f}")
print(f"  Upper Bound: {sales_upper:.2f}")
print(f"  Number of Outliers: {len(sales_outliers)} ({len(sales_outliers)/len(open_stores)*100:.2f}%)")

# Customer outliers
customer_outliers, cust_lower, cust_upper = detect_outliers_iqr(open_stores, 'Customers')
print(f"\nCustomer Outliers (Open Stores):")
print(f"  Lower Bound: {cust_lower:.2f}")
print(f"  Upper Bound: {cust_upper:.2f}")
print(f"  Number of Outliers: {len(customer_outliers)} ({len(customer_outliers)/len(open_stores)*100:.2f}%)")

# Visualize outliers
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].boxplot(open_stores['Sales'])
axes[0].set_title('Sales Box Plot (Open Stores)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Sales')
axes[0].axhline(sales_upper, color='red', linestyle='--', label='Upper Bound')
axes[0].axhline(sales_lower, color='red', linestyle='--', label='Lower Bound')
axes[0].legend()

axes[1].boxplot(open_stores['Customers'])
axes[1].set_title('Customers Box Plot (Open Stores)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Customers')
axes[1].axhline(cust_upper, color='red', linestyle='--', label='Upper Bound')
axes[1].axhline(cust_lower, color='red', linestyle='--', label='Lower Bound')
axes[1].legend()

plt.tight_layout()
plt.savefig('../docs/outliers_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 11. Data Quality Summary

In [ ]:
# Data quality report
quality_report = {
    'Total Records': len(train),
    'Unique Stores': train['Store'].nunique(),
    'Date Range': f"{train['Date'].min().date()} to {train['Date'].max().date()}",
    'Missing Values': train.isnull().sum().sum(),
    'Duplicate Rows': train.duplicated().sum(),
    'Zero Sales Records': (train['Sales'] == 0).sum(),
    'Closed Store Records': (train['Open'] == 0).sum(),
    'Records with Promo': (train['Promo'] == 1).sum(),
    'School Holiday Records': (train['SchoolHoliday'] == 1).sum(),
    'State Holiday Records': (train['StateHoliday'] != '0').sum()
}

print("\n=" * 60)
print("DATA QUALITY SUMMARY")
print("=" * 60)
for key, value in quality_report.items():
    print(f"{key:.<40} {value}")
print("=" * 60)

## 12. Key Findings and Recommendations

In [ ]:
findings = f"""
KEY FINDINGS FROM EXPLORATORY DATA ANALYSIS:

1. DATA QUALITY:
   - No missing values in the dataset
   - {len(train):,} total records covering {train['Store'].nunique()} stores
   - Date range: {train['Date'].min().date()} to {train['Date'].max().date()}
   - {(train['Sales'] == 0).sum():,} records with zero sales ({(train['Sales'] == 0).sum()/len(train)*100:.1f}%)

2. TARGET VARIABLE (Sales):
   - Mean: ${train['Sales'].mean():.2f}
   - Median: ${train['Sales'].median():.2f}
   - Highly right-skewed distribution (skewness: {train['Sales'].skew():.2f})
   - Log transformation recommended for modeling
   - Strong correlation with number of customers (r={train['Sales'].corr(train['Customers']):.3f})

3. TEMPORAL PATTERNS:
   - Clear weekly seasonality (higher sales mid-week)
   - Monthly patterns show peak sales in December
   - Day 7 (Sunday) shows reduced sales activity
   - Yearly trends indicate stable or growing sales

4. PROMOTIONAL EFFECTS:
   - Promo increases average sales by {((promo_sales[1]/promo_sales[0]-1)*100):.1f}%
   - {(train['Promo']==1).sum()/len(train)*100:.1f}% of records have active promotions
   - Promo also affects sales per customer metric

5. STORE HETEROGENEITY:
   - Significant variation in sales performance across stores
   - Top stores have >3x sales of bottom stores
   - Store-specific features will be crucial for modeling

6. CUSTOMER BEHAVIOR:
   - Average sales per customer: ${spc_data.mean():.2f}
   - Near-linear relationship between customers and sales
   - Customer count is highly predictive of sales

RECOMMENDATIONS FOR FEATURE ENGINEERING:

1. Temporal Features:
   - Day of week indicators
   - Month, quarter indicators
   - Weekend/weekday flags
   - Days to/from holidays
   - Week of year for seasonal patterns

2. Lag Features:
   - Previous day/week sales
   - Rolling averages (7-day, 30-day)
   - Year-over-year comparisons

3. Store Features:
   - Store average sales
   - Store sales volatility
   - Store-specific promo effects

4. Interaction Features:
   - Promo × Day of week
   - School holiday × Day of week
   - Store × Promo interactions

5. Target Encoding:
   - Store-level statistics
   - Time-period aggregations

6. Data Preprocessing:
   - Handle zero sales separately (closed stores)
   - Consider log transformation of target
   - Outlier treatment strategy needed
   - Train/validation split by time (time-series nature)
"""

print(findings)

# Save findings to file
with open('../docs/eda_key_findings.txt', 'w') as f:
    f.write(findings)

## Summary

This exploratory analysis has revealed:
- Clean dataset with no missing values
- Strong temporal patterns requiring time-based features
- Significant promotional effects on sales
- High store-level heterogeneity
- Clear relationship between customers and sales
- Need for log transformation of target variable

Next steps:
1. Feature engineering based on identified patterns
2. Additional store-level data integration
3. Time-series cross-validation strategy
4. Model selection and baseline establishment